In [ ]:
# Environment setup
const DEPENDENCIES = ["CairoMakie", "Gmsh", "HMatrices", "IterativeSolvers","LinearAlgebra", "LinearMaps", "SpecialFunctions", "GSL", "FMM3D", "FMM2D", "Meshes"];
import Pkg
Pkg.activate(temp=true)
Pkg.add(url="https://github.com/IntegralEquations/Inti.jl", rev="87f74e26e893ee275eb676e10a6470bf9aad9a48")
foreach(Pkg.add, DEPENDENCIES)

# Poisson solver

!!! note "Important points covered in this example"
      - Solving a volumetric problem
      - Using the method of manufactured solutions
      - ...

In [ ]:
using Inti

## Problem definition

In this example we will solve the Poisson equation in a domain $\Omega$ with
Dirichlet boundary conditions on $\Gamma := \partial \Omega$:
$$
  \begin{align}
      \Delta u &= f  \quad \text{in } \Omega \\
      u &= g  \quad \text{on } \partial \Gamma
  \end{align}
$$
where $f : \Omega \to \mathbb{R}$ and $g : \Gamma \to \mathbb{R}$ are given
functions.

Seeking for a solution $u$ of the form ...

In [ ]:
meshsize = 0.1

`n` in the VDIM paper

In [ ]:
interpolation_order = 3
qorder = Inti.Triangle_VR_interpolation_order_to_quadrature_order(interpolation_order)
nothing #hide

## Meshing

We now create the required meshes and quadratures for both $\Omega$ and $\Gamma$:

In [ ]:
using Gmsh # this will trigger the loading of Inti's Gmsh extension

function gmsh_disk(; name, meshsize, order = 1, center = (0, 0), paxis = (2, 1))
    try
        gmsh.initialize()
        gmsh.option.setNumber("General.Terminal", 0)
        gmsh.model.add("circle-mesh")
        gmsh.option.setNumber("Mesh.MeshSizeMax", meshsize)
        gmsh.option.setNumber("Mesh.MeshSizeMin", meshsize)
        gmsh.model.occ.addDisk(center[1], center[2], 0, paxis[1], paxis[2])
        gmsh.model.occ.synchronize()
        gmsh.model.mesh.generate(2)
        gmsh.model.mesh.setOrder(order)
        gmsh.write(name)
    finally
        gmsh.finalize()
    end
end

name = joinpath(@__DIR__, "disk.msh")
gmsh_disk(; meshsize, order = 2, name)

Inti.clear_entities!() # empty the entity cache
msh = Inti.import_mesh(name; dim = 2)
Ω = Inti.Domain(e -> Inti.geometric_dimension(e) == 2, Inti.entities(msh))
Γ = Inti.boundary(Ω)

Ωₕ = view(msh, Ω)
Γₕ = view(msh, Γ)

Use VDIM with the Vioreanu-Rokhlin quadrature rule

In [ ]:
Q = Inti.VioreanuRokhlin(; domain = :triangle, order = qorder);
dict = Dict(E => Q for E in Inti.element_types(Ωₕ))
Ωₕ_quad = Inti.Quadrature(Ωₕ, dict)
Γₕ_quad = Inti.Quadrature(Γₕ; qorder)

## Manufactured solution

For the purpose of comparing our numerical results to an exact solution, we
will use the method of manufactured solutions. For simplicity, we will take as
an exact solution

In [ ]:
uₑ = (x) -> cos(2 * x[1]) * sin(2 * x[2])

which yields

In [ ]:
fₑ = (x) -> -8 * uₑ(x)

Here is what the solution looks like:
qvals = map(Ωₕ_quad) do q
    return uₑ(q.coords)
end

ivals = Inti.quadrature_to_node_vals(Ωₕ_quad, qvals)

er = ivals - uₑ.(Ωₕ_quad.mesh.nodes)
norm(er,Inf)
Inti.write_gmsh_view(Ωₕ, uₑ.(Ωₕ.nodes))

## Boundary and integral operators

In [ ]:
using FMM2D

pde = Inti.Laplace(; dim = 2)

# Boundary operators
S_b2b, D_b2b = Inti.single_double_layer(;
    pde,
    target = Γₕ_quad,
    source = Γₕ_quad,
    compression = (method = :fmm, tol = 1e-12),
    correction = (method = :dim,),
)
S_b2d, D_b2d = Inti.single_double_layer(;
    pde,
    target = Ωₕ_quad,
    source = Γₕ_quad,
    compression = (method = :fmm, tol = 1e-12),
    correction = (method = :dim, maxdist = 5 * meshsize, target_location = :inside),
)

# Volume potentials
V_d2d = Inti.volume_potential(;
    pde,
    target = Ωₕ_quad,
    source = Ωₕ_quad,
    compression = (method = :fmm, tol = 1e-12),
    correction = (method = :dim, interpolation_order),
)
V_d2b = Inti.volume_potential(;
    pde,
    target = Γₕ_quad,
    source = Ωₕ_quad,
    compression = (method = :fmm, tol = 1e-12),
    correction = (
        method = :dim,
        maxdist = 5 * meshsize,
        interpolation_order,
        target_location = :on,
    ),
)

We can now solve a BIE for the unknown density $\sigma$:

In [ ]:
f = map(Ωₕ_quad) do q
    return fₑ(q.coords)
end
g = map(Γₕ_quad) do q
    return uₑ(q.coords)
end
rhs = V_d2b * f + g

using LinearAlgebra
L = -I / 2 + D_b2b

If `compression=none` or `compresion=hmatrix` is used above for constructing `D_b2b`, we could alternately use dense linear algebra:

In [ ]:
#F = lu(L)
#σ = F \ rhs

using IterativeSolvers
σ, hist =
    gmres(L, rhs; log = true, abstol = 1e-10, verbose = false, restart = 100, maxiter = 100)
@show hist

To check the solution, lets evaluate it at the nodes $\Omega$

In [ ]:
uₕ_quad = -(V_d2d * f) + D_b2d * σ
uₑ_quad = map(q -> uₑ(q.coords), Ωₕ_quad)
er = abs.(uₕ_quad - uₑ_quad)
@assert norm(er) < 1e-5 #hide
@show norm(er, Inf)

## Visualize the solution error using Gmsh

In [ ]:
er_nodes = Inti.quadrature_to_node_vals(Ωₕ_quad, er)
uₕ_nodes = Inti.quadrature_to_node_vals(Ωₕ_quad, uₕ_quad)

using GLMakie, Meshes
fig = Figure(; size = (1200, 400))
ax1 = Axis(fig[1, 1]; aspect = DataAspect(), title = "solution")
colorrange = extrema(uₕ_nodes)
viz!(Ωₕ; showsegments = true, color = uₕ_nodes, colorrange)
Colorbar(fig[1, 2]; colorrange = colorrange)
ax2 = Axis(fig[1, 3]; aspect = DataAspect(), title = "error")
colorrange = extrema(er_nodes)
viz!(Ωₕ; showsegments = false, color = er_nodes, colorrange)
Colorbar(fig[1, 4]; colorrange = colorrange)
fig

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*